In [6]:
import pandas as pd
import numpy as np
import nltk
import re

In [7]:
5+6

11

In [8]:
import nltk
nltk.__version__

'3.8.1'

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB


In [10]:
# df = pd.read_csv("training.csv", encoding="ISO-8859-1")
df = pd.read_csv("training.csv", header=None, names=["target", "unique_id", "date", "flag", "user_id", "text"], encoding="ISO-8859-1")
df.sample(10)

,target,unique_id,date,flag,user_id,text
273623,0,1990437839,Mon Jun 01 04:52:14 PDT 2009,NO_QUERY,auntieflo,"to do: dark + light + bedding wash, trinity ap..."
771230,0,2302287583,Tue Jun 23 16:35:53 PDT 2009,NO_QUERY,JennyBoBennyy,Damn if its raining in the city its probably g...
418326,0,2061740580,Sat Jun 06 21:30:40 PDT 2009,NO_QUERY,realchrissystar,Really bad night for Grams ..Got a few cracke...
1322517,4,2014803277,Wed Jun 03 02:44:23 PDT 2009,NO_QUERY,raquelaberakiki,Photo: just wanna show you my pink guitar.. h...
487001,0,2181871341,Mon Jun 15 12:24:41 PDT 2009,NO_QUERY,HeroesJonas7495,"http://twitpic.com/7hhd0 - Ok, this is my favo..."
1032160,4,1933161370,Tue May 26 23:09:24 PDT 2009,NO_QUERY,mhaemee,@faye42397 thanks for following me .
13529,0,1553295553,Sat Apr 18 13:53:13 PDT 2009,NO_QUERY,Ciel_Rouge,Stuck in traffic on the Cross Bronx
1158501,4,1979271225,Sun May 31 02:20:04 PDT 2009,NO_QUERY,BoxRoom,"@revoltingross Oop, missed this message. Yes, ..."
1202955,4,1985947132,Sun May 31 18:04:25 PDT 2009,NO_QUERY,tinkhanson,@jennyquotes Thanks for the great suggestions!...
1206776,4,1986881327,Sun May 31 19:27:16 PDT 2009,NO_QUERY,desmondngai,@cafedavid me need more kgs &amp; muscles too!


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   target     1600000 non-null  int64 
 1   unique_id  1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   flag       1600000 non-null  object
 4   user_id    1600000 non-null  object
 5   text       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [12]:
df.shape

(1600000, 6)

In [13]:
df.drop(['unique_id','date','flag','user_id'], axis = 1, inplace = True) 

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   text    1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [15]:
df.sample(10)

,target,text
19136,0,standing outside in the cold
1306987,4,@myrtlebeachblog Meh
694878,0,Tried doing another 10Km jog 2day but managed ...
23417,0,I can't sleep I can't stop thinking about wha...
329938,0,Knees still hurt
1031424,4,@Aashay ya ppl are really warm/open there. whe...
853172,4,Morning all and what a glorious one it is in M...
495515,0,wish i could have curly hair. oh wish i could....
1324851,4,The gas station was closed. So i peed by the d...
1251324,4,hot hot hotttttttttttttttttttttttttttttttttttt...


In [16]:
df.rename(columns={'target': 'sentiment', 'text': 'tweet'}, inplace = True)

In [17]:
df.sample(10)

,sentiment,tweet
529591,0,this sucks. i'm stuck at home. i really need t...
1327704,4,#Sims 3: Lots of fun. Amazingly flexible item...
1562414,4,@ashleytisdale Yesssss Of Course! Baltimore MD...
647105,0,@so_hailish me too..:/ music school in the aft...
620902,0,sleepy... i wish i could take a nap.
36089,0,Up and at it early...work tonight
689725,0,Uh oh got a caffiene headace too tired now
748176,0,2nd day of summer and I need to turn on the he...
469664,0,hope it doesnt thunder today but its meant 2
1267055,4,"@officialTila Well, that's something to be pro..."


In [18]:
df = df.dropna()
df.sample(10)

,sentiment,tweet
110625,0,i missed merlin again
135900,0,@starafar thanks pat. ive tried ripping it wit...
931558,4,"@hma4983 You are welcome, lady!"
728803,0,3 more hours 2 go.
1430611,4,got a picnic to go to tomoz!!!!!!!!! listenin...
427515,0,"Hearing dark, dark rumours about the #f1 this ..."
1498388,4,@liteNup http://twitpic.com/6vh79 - not in a m...
1252376,4,@sontortillas get a 450d (rebel xsi) entry le...
1587180,4,Exchange SmÃ¸rrebrÃ¸d with Tapas
742539,0,@WeatherKim Sorry I couldn't tell you and Angi...


In [19]:
print(df.info())
print("The Shape Of The Dataset Is :",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   tweet      1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB
None
The Shape Of The Dataset Is : (1600000, 2)


In [20]:
df['sentiment'] = df['sentiment'].replace({0: 'negative', 4: 'positive'})

In [21]:
df.sample(5)

,sentiment,tweet
401724,negative,Why did YouTube suspend the Jonas Brothers' ac...
586974,negative,@kencarr Poor baby I hope you feel better. ...
1058200,positive,@alexbrannan 'get twitter.com and follow me @a...
417143,negative,@paulinahearts haha don't even whatt??? btw yo...
394702,negative,"@bethbabezz lmao beth, nope , well not yet.. ..."


In [22]:
# Remove URLs, special characters, punctuation, and numbers from tweet column
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[0-9]+', '', x))


In [23]:
print(df.sample(10))
print('\033[1m' + 'The information of the dataset is :' + '\033[0m')
print(df.info())
print('\033[1m' + 'The Shape Of The Dataset Is :' + '\033[0m',df.shape)

        sentiment                                              tweet
810067   positive  Goes without saying Funnest night of my life j...
921482   positive                                  amalinaaa what  x
212895   negative  heybabeitsnikki been there done that i cried e...
381955   negative                 geeshadowsxo oh dont cry please  x
18693    negative                      You make me wanna cry iPhone 
1059818  positive  Wow In three weeks Im flying to Rome Im so exc...
666635   negative                                   charcoalfeather 
659429   negative  I pulled the battery out and it still wont upl...
171580   negative  nickjfrost thats what i tell everyone their fo...
279008   negative    nbonsack haha yes I dont have any of there CDs 
The information of the dataset is :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentim

In [24]:
df['tweet'] = df['tweet'].str.lower()

In [25]:
df.sample(10)

,sentiment,tweet
1081125,positive,at edwards cinema in lb with richiedigs
853271,positive,i think were all lookin forward to hearing it...
649510,negative,uhohhippos and i miss you too i found somethin...
534673,negative,my internet has been down for ever a beloved h...
374215,negative,hthemovie halloween got banned in thailand su...
838547,positive,quotthe future of foodquot is on hulucom go ch...
687751,negative,i need followers tweetfakes
1142501,positive,bostonwriter stop quit become a chef the hilar...
437054,negative,janelle i ate her flowers she is more patien...
222131,negative,ouchyyy booo headache going to beed right now...


In [26]:
# stop_words = set(stopwords.words('english'))
# df['tweet'] = df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))